In [1]:
# default_exp models

## Imports

In [2]:
# EXPORT
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from surrogates4sims.utils import printNumModelParams

## Models

### Deep Fluids Models

In [3]:
# EXPORT 
class convBlock(nn.Module):
    def __init__(self,num_conv=4,in_channels=128,filters=128,act=nn.LeakyReLU(),downSample=True):
        super(convBlock,self).__init__()
        self.num_conv = num_conv
        self.in_channels = in_channels
        self.act = act
        
        layers = []
        for i in range(num_conv):
            if i == 0:
                layers.append(nn.Conv2d(in_channels, filters, kernel_size=3, stride=1,padding=1))
                layers.append(nn.BatchNorm2d(filters))
                layers.append(act)
            else:
                layers.append(nn.Conv2d(filters, filters, kernel_size=3, stride=1,padding=1))
                layers.append(nn.BatchNorm2d(filters))
                layers.append(act)
        self.convs = nn.Sequential(*layers)
        #self.downSampleLayer = nn.Conv2d(in_channels+filters, in_channels+filters,kernel_size=3,stride=2,padding=1)
        self.downSampleLayer = nn.Conv2d(filters, filters,kernel_size=3,stride=2,padding=1)

    def forward(self,x):
        #print(x.shape)
        x0 = x
        x = self.convs(x)
        #x = torch.cat([x,x0],axis=1)
        x = self.downSampleLayer(x)
        x = torch.cat([x,F.interpolate(x0,scale_factor=.5)],axis=1)
        return x
    
class convTransBlock(nn.Module):
    def __init__(self, num_conv=4, in_channels=128, filters=128, act=nn.LeakyReLU(), skip_connection=False, stack=False):
        super(convTransBlock,self).__init__()
        self.filters = filters
        self.num_conv = num_conv
        self.in_channels = in_channels
        self.act = act
        self.skip_connection = skip_connection
        self.stack = stack
        self.upsample = torch.nn.modules.Upsample(scale_factor=2)
        layers = []
        for i in range(num_conv-1):
            if i == 0:
                layers.append(nn.ConvTranspose2d(in_channels,out_channels=filters, kernel_size=3, stride=1,padding=1))
                layers.append(nn.BatchNorm2d(filters))
                layers.append(act)
            else:
                layers.append(nn.ConvTranspose2d(filters,out_channels=filters, kernel_size=3, stride=1,padding=1))
                layers.append(nn.BatchNorm2d(filters))
                layers.append(act)                
        layers.append(nn.ConvTranspose2d(filters, out_channels=filters, kernel_size=3, stride=2,
                                         padding=1, output_padding=1))
        layers.append(act)
        self.seq = nn.Sequential(*layers)
    
    def forward(self,x):
        x0 = x
        x = self.seq(x)
        if self.skip_connection:
            x += self.upsample(x0)
        if self.stack:
            x = torch.cat([x, self.upsample(x0)], axis=1)
        #print(x.shape)
        return x
    
class Generator(nn.Module):
    def __init__(self,z, filters, output_shape,
                 num_conv=4, conv_k=3, last_k=3, repeat=0, 
                 skip_connection=False, act=nn.LeakyReLU(),stack=False):
        super(Generator,self).__init__()
        if repeat == 0:
            repeat_num = int(np.log2(torch.max(output_shape[1:]))) - 2
        else:
            repeat_num = repeat
        x0_shape = [filters] + [int(i/np.power(2, repeat_num-1)) for i in output_shape[1:]]
        self.x0_shape = x0_shape
        self.output_shape = output_shape
        self.filters = filters
        self.num_conv = num_conv
        num_output = int(np.prod(x0_shape))
        self.num_output = num_output
        self.linear = nn.Linear(z.shape[1], num_output)
        
        convTransBlockLayers = []
        ch = filters
        for i in range(repeat_num-1):
            #print(ch)
            convTransBlockLayers.append(convTransBlock(num_conv, ch, filters, act, skip_connection, stack))
            if stack:
                ch += filters

        self.convTransBlockLayers = nn.Sequential(*convTransBlockLayers)
        if ch > filters:
            n = ch
        else:
            n = filters
        self.lastConv = nn.Conv2d(n,int(output_shape[0]),kernel_size=3, stride=1,padding=1)
        self.skip_connection = skip_connection
        self.stack = stack

    def forward(self, x):
        x = self.linear(x)
        x = x.view(-1,self.x0_shape[0],self.x0_shape[1],self.x0_shape[2])
        x = self.convTransBlockLayers(x)
        x = self.lastConv(x)
        return x
    
class Encoder(nn.Module):
    def __init__(self, x, filters, z_num, num_conv=4, conv_k=3, repeat=0, act=nn.LeakyReLU()):
        super(Encoder,self).__init__()
        
        x_shape = x.shape[1:]
        if repeat == 0:
            repeat_num = int(np.log2(np.max(x_shape[1:]))) - 2
        else:
            repeat_num = repeat
        
        self.filters = filters
        self.act = act
        self.conv1 = nn.Conv2d(x_shape[0], filters, kernel_size=conv_k, stride=1,padding=1)
        
        ch = filters
        convLayers = []
        for idx in range(0,repeat_num):
            convLayers.append(convBlock(num_conv, ch, filters, act=nn.LeakyReLU(), downSample=True))
            ch += filters
            
        self.convs = nn.Sequential(*convLayers)
        h = [i//2**repeat_num for i in x_shape[1:]]
        self.nLinearInput = (ch)*np.prod(h)
        self.linear = nn.Linear(self.nLinearInput,z_num)
                             
    def forward(self,x):
        x = self.act(self.conv1(x))
        #print(x.shape)
        x = self.convs(x)
        #print(x.shape)
        x = x.view(x.size(0),-1)
        x = self.linear(x)
        return x

class AE_no_P(nn.Module):
    def __init__(self, encoder,generator):
        super(AE_no_P,self).__init__()
        self.encoder = encoder
        self.generator = generator
        
    def forward(self,x):
        x = self.encoder(x)
        #x = torch.cat([x,p],axis=1)
        x = self.generator(x)
        return x
    
class AE_xhat_z(nn.Module):
    def __init__(self, encoder,generator):
        super(AE_xhat_z,self).__init__()
        self.encoder = encoder
        self.generator = generator
        
    def forward(self,x):
        z = self.encoder(x)
        x = self.generator(z)
        return x, z
    
class AE_xhat_zV2(nn.Module):
    def __init__(self, X, filters=32, latentDim=16, num_conv=2, repeat=0, 
                 skip_connection=False, stack=False, conv_k=3, last_k=3,
                 act=nn.LeakyReLU, return_z=True, stream=True, device='cpu'):
        super(AE_xhat_zV2,self).__init__()
        
        self.filters = filters
        self.latentDim = latentDim
        self.num_conv = num_conv
        self.repeat = repeat
        self.skip_connection = skip_connection
        self.stack = stack
        self.act = act
        self.conv_k = 3
        self.last_k = 3
        self.device = device
        self.return_z = return_z
        self.stream = stream
        
        self.encoder = Encoder(X,filters,latentDim,num_conv=num_conv).to(device)
        
        z = self.encoder(X)
        
        if stream:
            self.output_shape = torch.tensor(X[0][1:].shape)
        else:
            self.output_shape = torch.tensor(X[0].shape)

        self.generator = Generator(z, filters, self.output_shape,
                                   num_conv, conv_k, last_k, repeat, 
                                   skip_connection, act=act, stack=stack).to(device)
        
    def forward(self,x):
        z = self.encoder(x)
        x = self.generator(z)
        if self.return_z:
            return x, z
        else:
            return x

#### Playground

In [4]:
bz = 8
latentDim = 16
filters = 128
num_conv = 4
num_Gconv= 128

In [5]:
X = torch.randn([bz, 2, 128, 96]) #, torch.Size([32, 3]))
X.shape

torch.Size([8, 2, 128, 96])

In [6]:
output_shape = torch.tensor(X[0].shape)
output_shape

tensor([  2, 128,  96])

In [7]:
p = torch.rand([bz,3])
p.shape

torch.Size([8, 3])

In [8]:
C = convBlock(num_conv=num_conv,in_channels=X.shape[1],filters=filters)
C

convBlock(
  (act): LeakyReLU(negative_slope=0.01)
  (convs): Sequential(
    (0): Conv2d(2, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
  )
  (downSampleLayer): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
)

In [9]:
out = C(X)
out.shape

torch.Size([8, 130, 64, 48])

In [10]:
CTB = convTransBlock(num_conv=num_conv, filters=128, act=nn.LeakyReLU(), skip_connection=False, stack=True)
XX = torch.randn([bz, 128,8,6]) #, torch.Size([32, 3]))
XX.shape

torch.Size([8, 128, 8, 6])

In [11]:
out = CTB(XX)
out.shape

torch.Size([8, 256, 16, 12])

##### Encoder

In [12]:
E = Encoder(X,filters,latentDim,num_conv=num_conv)
E

Encoder(
  (act): LeakyReLU(negative_slope=0.01)
  (conv1): Conv2d(2, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (convs): Sequential(
    (0): convBlock(
      (act): LeakyReLU(negative_slope=0.01)
      (convs): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01)
        (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): LeakyReLU(negative_slope=0.01)
        (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (8): LeakyReLU(negative_slope=0.01)
        (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (10): BatchNor

In [13]:
printNumModelParams(E)

94 layers require gradients (unfrozen) out of 94 layers
5,319,184 parameters require gradients (unfrozen) out of 5,319,184 parameters


In [14]:
out = E(X)
out.shape

torch.Size([8, 16])

In [15]:
output_shape = torch.tensor(X.shape[1:])
output_shape

tensor([  2, 128,  96])

##### Generator

In [16]:
z = torch.randn(bz, latentDim + p.shape[1])
z.shape

torch.Size([8, 19])

In [17]:
G = Generator(z, 128, output_shape,
                 num_conv=2, conv_k=3, last_k=3, repeat=0, 
                 skip_connection=False, act=nn.LeakyReLU(),stack=True)
G

Generator(
  (linear): Linear(in_features=19, out_features=6144, bias=True)
  (convTransBlockLayers): Sequential(
    (0): convTransBlock(
      (act): LeakyReLU(negative_slope=0.01)
      (upsample): Upsample(scale_factor=2.0, mode=nearest)
      (seq): Sequential(
        (0): ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01)
        (3): ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
        (4): LeakyReLU(negative_slope=0.01)
      )
    )
    (1): convTransBlock(
      (act): LeakyReLU(negative_slope=0.01)
      (upsample): Upsample(scale_factor=2.0, mode=nearest)
      (seq): Sequential(
        (0): ConvTranspose2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [18]:
printNumModelParams(G)

28 layers require gradients (unfrozen) out of 28 layers
2,200,834 parameters require gradients (unfrozen) out of 2,200,834 parameters


In [19]:
out = G(z)
out.shape

torch.Size([8, 2, 128, 96])

##### AE

In [20]:
# EXPORT
class GeneratorOld(nn.Module):
    def __init__(self,z, filters, output_shape,
                 num_conv=4, conv_k=3, last_k=3, repeat=0, 
                 skip_connection=False, act=nn.LeakyReLU()):
        super(GeneratorOld,self).__init__()
        if repeat == 0:
            repeat_num = int(np.log2(torch.max(output_shape[1:]))) - 2
        else:
            repeat_num = repeat
        x0_shape = [filters] + [int(i/np.power(2, repeat_num-1)) for i in output_shape[1:]]
        self.x0_shape = x0_shape
        num_output = int(np.prod(x0_shape))
        self.linear = nn.Linear(z.shape[1], num_output)
        convTransBlockLayers = []
        for i in range(repeat_num-1):
            convTransBlockLayers.append(convTransBlock(num_conv,filters,act,skip_connection))
        self.convTransBlockLayers = nn.Sequential(*convTransBlockLayers)
        self.lastConv = nn.Conv2d(filters,int(output_shape[0]),kernel_size=3, stride=1,padding=1)

    def forward(self, x):
        x = self.linear(x)
        x = x.view(-1,self.x0_shape[0],self.x0_shape[1],self.x0_shape[2])
        x = self.convTransBlockLayers(x)
        #x = self.lastConv(x)
        return x

In [21]:
E = Encoder(X,filters,latentDim,num_conv=num_conv)
G = Generator(z, num_Gconv, output_shape,
                 num_conv=num_conv, conv_k=3, last_k=3, repeat=0, 
                 skip_connection=False, act=nn.LeakyReLU())

model = AE(E,G)

NameError: name 'AE' is not defined

In [ ]:
Xhat = model(X,p)
Xhat.shape

In [ ]:
# Loss Function
loss_func = torch.nn.MSELoss()
loss_func

In [ ]:
mseVal = loss_func(X,model(X,p))
mseVal

### Build AE_no_P

In [22]:
z = E(X)
z.shape

torch.Size([8, 16])

In [23]:
G = Generator(z, 128, output_shape,
                 num_conv=2, conv_k=3, last_k=3, repeat=0, 
                 skip_connection=False, act=nn.LeakyReLU(),stack=True)
G

Generator(
  (linear): Linear(in_features=16, out_features=6144, bias=True)
  (convTransBlockLayers): Sequential(
    (0): convTransBlock(
      (act): LeakyReLU(negative_slope=0.01)
      (upsample): Upsample(scale_factor=2.0, mode=nearest)
      (seq): Sequential(
        (0): ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01)
        (3): ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
        (4): LeakyReLU(negative_slope=0.01)
      )
    )
    (1): convTransBlock(
      (act): LeakyReLU(negative_slope=0.01)
      (upsample): Upsample(scale_factor=2.0, mode=nearest)
      (seq): Sequential(
        (0): ConvTranspose2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [24]:
model = AE_no_P(E,G)

In [25]:
out = model(X)
out.shape

torch.Size([8, 2, 128, 96])

### Build AE_xhat_z

In [26]:
m = AE_xhat_z(E,G)
m

AE_xhat_z(
  (encoder): Encoder(
    (act): LeakyReLU(negative_slope=0.01)
    (conv1): Conv2d(2, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (convs): Sequential(
      (0): convBlock(
        (act): LeakyReLU(negative_slope=0.01)
        (convs): Sequential(
          (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): LeakyReLU(negative_slope=0.01)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): LeakyReLU(negative_slope=0.01)
          (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (8): LeakyReLU(negative_slope=0.01)
          (9): Conv2d(128, 128, kernel_size=(3, 3

In [27]:
xhat, z = m(X)

In [28]:
xhat.shape, z.shape

(torch.Size([8, 2, 128, 96]), torch.Size([8, 16]))